In [750]:
from urllib.request import urlopen as uReq
import re
import requests
import bs4
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
from os.path import basename

In [751]:
width_dict = {"max-width:7%;": "1", "max-width:14%;": "2", "max-width:21%;": "3", 
              "max-width:28%;": "4", "max-width:35%;": "5"} #1 being least 5 being most

In [752]:
gender_dict = {"Homme": "Male", "Femme": "Female"}

In [753]:
disease_dict = {"Acné": "Acne", "Allergie":"Allergy", "Allergie acariens":"Dust mite allergy", "Asthme":"Asthma",
               "Ampoules - cloques":"Blisters - blisters", "Angine de poitrine":"Angina pectoris",
               "Angoisse / trouble panique":"Anxiety", "Arrêter de fumer":"Stop smoking", "Bronchite":"Bronchitis",
               "Arthrite psoriatique":"Psoriatic arthritis", "Arthrose":"Osteoarthritis", "Brûlures":"Burns",
               "Arthrose colonne vertébrale":"Spine osteoarthritis", "Cancer du sein":"Breast cancer",
               "Céphalées":"Headache", "Cholestérol élevé":"High cholesterol", "Couperose":"Rosacea",
               "Colite ulcéreuse":"Ulcerative colitis", "Crise cardiaque":"Heart attack", "Tremblements":"Tremors",
               "Complications post-opératoires":"Post-operative complications", "Démangeaison":"Itching",
               "Cycle menstruel irrégulier":"Irregular menstrual cycle", "Défaillance nerveuse":"Nerve failure",
               "Dépression":"Depression", "Dépression post-natale":"Post-natal depression", "Thyroïde":"Thyroid",
               "Dépression chronique":"Depression", "Diabète type 1":"Type 1 diabetes", "TDAH":"ADHD",
               "Diabète type 2":"Type 2 diabetes", "Diabète type 3":"Type 3 diabetes", "Urticaire":"Urticaria",
               "Troubles d'angoisse":"Anxiety", "Trouble stress post-traumatique":"Post-traumatic stress disorder",
               "Trouble schizotypique":"Schizotypal disorder", "Trouble rythme cardiaque":"Heart rhythm disorder",
               "Trouble obsessionnel compulsif (TOC)":"Obsessive Compulsive Disorder (OCD)", "Psychose":"Psychosis",
               "Trouble bipolaire / trouble mani dépressif":"Bipolar disorder / manic depressive disorder",
               "Thyroïde enlevée":"Thyroid removed", "Tension élevée":"High tension", "Sinusite":"Sinusitis",
               "Spondylarthrite ankylosante":"Ankylosing spondylitis", "Schizophrénie":"Schizophrenia",
               "Rhume des foins":"Hay fever", "Rhumatisme":"Rheumatism", "Retour d'âge":"Return of age",
               "Pyélonéphrite":"Pyelonephritis", "Problèmes psychiques":"Psychic problems", "Otite":"Otitis",
               "Problèmes cardiaques":"Heart problems", "Pouls élevé":"High pulse", "Pneumonie":"Pneumonia",
               "Polyneuropathie":"Polyneuropathy", "Phobie sociale":"Social phobia", "Narcolepsie":"Narcolepsy",
               "MST (maladies sexuellement transmissibles)":"STDs (sexually transmitted diseases)",
               "Mise en place stérilet":"IUD insertion", "Maladie de Lyme - borréliose":"Lyme disease - borreliosis",
               "Maladie de Graves":"Graves disease", "Maladie de Crohn":"Crohn's disease", "Insomnie":"Insomnia",
               "Maladie d'Hashimoto":"Hashimoto's disease", "Maladie auto-immunitaire":"Autoimmune disease",
               "Mal de gorge":"Sore throat", "Inflammation articulaire":"Joint inflammation", "Hernie":"Hernia",
               "Infection urinaire":"Urinary tract infection", "Infection gencive":"Gum infection", "Fatigue":"Tired",
               "Infection des amygdales":"Infection of the tonsils", "Infection de la prostate":"Prostate infection",
               "Infection de la machoire":"Jaw infection", "Infection de l'estomac":"Stomach infection",
               "Infection dans la poitrine":"Chest Infection", "Infection bactérienne":"Bacterial infection",
               "Infection (plaie)":"Infection (wound)", "Glande thyroïde lente":"Slow thyroid gland",
               "Glande thyroïde hyperactive":"Overactive thyroid gland", "Fibromyalgie":"Fibromyalgia",
               "Épisode maniaque":"Manic episode", "Épilepsie":"Epilepsy", "Douleur, général":"Pain, general",
               "Douleurs menstruation":"Menstruation pain", "Douleur articulaire":"Articular pain",
               "Douleur nerveuse":"Nerve pain", "Hypothyroïdie":"Hypothyroidism", "Battements de coeur": "Heartbeats",
               "Ablation de la thyroïde":"Thyroid removal", "Pas dans la liste":"Other", "Apnée":"Apnea",
               "Tendinite épaule":"Shoulder tendinitis", "Alcoolisme":"Alcoholism", "Mal aux dents":"Toothache",
               "Syndrome des jambes sans repos":"Restless legs syndrome", "Vertige - tournis":"Vertigo - spinning",
               "Cholestérol trop élevé":"High cholesterol","Crampes jambes":"Leg cramps", "Acidité":"Acidity",
               "Effet indésirable d'un autre médicament":"Side effect of another medicine", "Attaque":"Attack",
               "Anémie suite déficience vitamine B12":"Anemia", "Défaillances cardiaques":"Cardiac failures",
               "Brûlure d'estomac - acidité gastrique":"Heartburn - gastric acidity", "Sarcoïdose":"Sarcoidosis",
               "Arthrite chronique juvénile":"Juvenile chronic arthritis","Endométriose":"Endometriosis",
               "Angoisses":"Anxiety","Trouble d'angoisse":"Anxiety", "Bouche sèche":"Dry Mouth",
               "Trouble dysthémie avec TDA":"Dysthemia disorder with ADD"}

In [754]:
#grouped anxiety together
#some mnot translated like migraine and contraception
#grouped high cholesterol and too high cholesterol
#grouped depression

In [755]:
#pd.cut(age_range, bins=4, labels=False)

# 1. Collecting Data: webscrapping
In order to obtain all the reviews for the different medicine, I scrapped the meamedica website. In order to analyse and proceed to text mine each medicine separatly later on, I decided to create one dataset for each medicine, all have the exact same format.

## 1.1. Collecting Levothyrox

In [756]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/glande-thyroide-hypothyroidie-a-action-lente/levothyrox'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [757]:
#We collect all pages for Levothyrox

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [758]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [759]:
df.to_csv('levothyrox_review.csv') #we store the dataframe in a csv file #use url to define file name

## 1.2. Collecting Mirena

In [760]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/contraception-autre/mirena'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [761]:
#We collect all pages for Mirena

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [762]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [763]:
df.to_csv('mirena_review.csv') #we store the dataframe in a csv file #use url to define file name

## 1.3. Collecting Tramadol

In [764]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/douleurs-morphine/tramadol'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [765]:
#We collect all pages for Tramadol

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [766]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [767]:
df.to_csv('tramadol_review.csv') #we store the dataframe in a csv file

## 1.4. Collecting Paroxetine

In [768]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-irs/paroxetine'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [769]:
#We collect all pages for Paroxetine

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [770]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [771]:
df.to_csv('paroxetine_review.csv') #we store the dataframe in a csv file

## 1.5. Collecting Effexor

In [772]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-autre/effexor'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [773]:
#We collect all pages for Effexor

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information for 1st element of subText
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [774]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [775]:
df.to_csv('effexor_review.csv') #we store the dataframe in a csv file

## 1.6. Collecting Champix

In [776]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/toxicomanie/champix'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [777]:
#We collect all pages for Champix

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [778]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [779]:
df.to_csv('champix_review.csv') #we store the dataframe in a csv file

## 1.7. Collecting Lyrica

In [780]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/epilepsie/lyrica'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [781]:
#We collect all pages for Lyrica

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [782]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [783]:
df.to_csv('lyrica_review.csv') #we store the dataframe in a csv file

## 1.8. Collecting Simvastatine

In [784]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/cholesterol/simvastatine'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [785]:
#We collect all pages for Simvastine

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [786]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [787]:
df.to_csv('simvastine_review.csv') #we store the dataframe in a csv file

## 1.9. Collecting Amoxicilline

In [788]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/antibiotiques-penicillines-a-large-spectre/amoxicilline'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [789]:
#We collect all pages for Amoxicilline

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [790]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [791]:
df.to_csv('amoxicilline_review.csv') #we store the dataframe in a csv file

## 1.10. Collecting Sertraline

In [792]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-irs/sertraline'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [793]:
#We collect all pages for Sertraline

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [794]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [795]:
df.to_csv('sertraline_review.csv') #we store the dataframe in a csv file

## 1.11. Collecting Cymbalta

In [796]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-autre/cymbalta'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [797]:
#We collect all pages for Cymbalta

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [798]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [799]:
df.to_csv('cymbalta_review.csv') #we store the dataframe in a csv file

## 1.12. Collecting Seroplex

In [800]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-irs/seroplex'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [801]:
#We collect all pages for Seroplex

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [802]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [803]:
df.to_csv('seroplex_review.csv') #we store the dataframe in a csv file

## 1.13. Collecting Crestor

In [804]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/cholesterol/crestor'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [805]:
#We collect all pages for Crestor

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [806]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [807]:
df.to_csv('crestor_review.csv') #we store the dataframe in a csv file

## 1.14. Collecting Tamoxifene

In [808]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/cancer-hormones-et-antihormones/tamoxifene'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [809]:
#We collect all pages for Tamoxifene

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [810]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [811]:
df.to_csv('tamoxifene_review.csv') #we store the dataframe in a csv file

## 1.15. Collecting Citalopram

In [812]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-irs/citalopram'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [813]:
#We collect all pages for Citalopram

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [814]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [815]:
df.to_csv('citalopram_review.csv') #we store the dataframe in a csv file

## 1.16. Collecting Metformine

In [816]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/diabetes-medicaments-oraux/metformine'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [817]:
#We collect all pages for Metformine

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [818]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [819]:
df.to_csv('metformine_review.csv') #we store the dataframe in a csv file

## 1.17. Collecting Deroxat

In [820]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-irs/deroxat'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [821]:
#We collect all pages for Deroxat

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [822]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [823]:
df.to_csv('deroxat_review.csv') #we store the dataframe in a csv file

## 1.18. Collecting Tahor

In [824]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/cholesterol/tahor'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [825]:
#We collect all pages for Tahor

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [826]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [827]:
df.to_csv('tahor_review.csv') #we store the dataframe in a csv file

## 1.19. Collecting Victoza

In [828]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/diabetes-medicaments-oraux/victoza'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [829]:
#We collect all pages for Victoza

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)


In [830]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [831]:
df.to_csv('victoza_review.csv') #we store the dataframe in a csv file

## 1.20. Collecting Propranolol

In [832]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/tension-arterielle-beta-bloquant/propranolol'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [833]:
#We collect all pages for Propranolol

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [834]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [835]:
df.to_csv('propranolol_review.csv') #we store the dataframe in a csv file

## 1.21. Collecting Abilify

In [836]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/psychose-schizophrenie-antipsychotique/abilify'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [837]:
#We collect all pages for Abilify

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [838]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [839]:
df.to_csv('abilify_review.csv') #we store the dataframe in a csv file

## 1.22. Collecting Concerta

In [840]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/adhd-psychostimulants/concerta'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [841]:
#We collect all pages for Concerta

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [842]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [843]:
df.to_csv('concerta_review.csv') #we store the dataframe in a csv file

## 1.23. Collecting Keppra

In [844]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/epilepsie/keppra'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [845]:
#We collect all pages for Keppra

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [846]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [847]:
df.to_csv('keppra_review.csv') #we store the dataframe in a csv file

## 1.24. Collecting Roaccutane

In [848]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/acne/roaccutane'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [849]:
#We collect all pages for Roaccutane

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [850]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [851]:
df.to_csv('roaccutane_review.csv') #we store the dataframe in a csv file

## 1.25. Collecting Lexapro

In [852]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-irs/lexapro'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [853]:
#We collect all pages for Lexapro

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [854]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [855]:
df.to_csv('lexapro_review.csv') #we store the dataframe in a csv file

## 1.26. Collecting Aerius

In [856]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/allergies/aerius'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [857]:
#We collect all pages for Aerius

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [858]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [859]:
df.to_csv('aerius_review.csv') #we store the dataframe in a csv file

## 1.27. Collecting Valdoxan

In [860]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/depression-antidepresseurs-autre/valdoxan'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [861]:
#We collect all pages for Valdoxan

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [862]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [863]:
df.to_csv('valdoxan_review.csv') #we store the dataframe in a csv file

## 1.28. Collecting Cerazette

In [864]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/contraception-autre/cerazette'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [865]:
#We collect all pages for Cerazette

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [866]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [867]:
df.to_csv('cerazette_review.csv') #we store the dataframe in a csv file

## 1.29. Collecting Doxycycline

In [868]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/antibiotiques-tetracyclines/doxycycline'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [869]:
#We collect all pages for Doxycycline

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [870]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [871]:
df.to_csv('doxycycline_review.csv') #we store the dataframe in a csv file

## 1.30. Collecting Methotrexate

In [872]:
#we collect the urls 1-7 because other pages are not available
url = 'https://www.meamedica.fr/systeme-immunitaire-immunosuppression/methotrexate'
url = [url] + [url + '/' + str(i) for i in range(2, 8)]

In [873]:
#We collect all pages for Methotrexate

rx_dose = re.compile('\((.*)\)')

rows = []
for u in url:
    html = requests.get(u).text
    soup = BeautifulSoup(html, "html.parser")
    for div_tag in soup.findAll('div', {'class': 'vote rounded btn-left'}): 
#note: there are some of these div_tag that contain boxes of text we do not want
        row = {}
        h3_tag = div_tag.find('h3')
        if h3_tag is None:
            continue #removes all the irrelevant <div class>:'vote rounded btn-left'
        row['medicine'] = h3_tag.text.strip() #Picks up medicine name and assigns to row of that name
        subtext_tag = div_tag.find('div', {'class': 'subText'}) 
        #div_tag contains multiple elements we want to store in different columns
        subText = subtext_tag.text.split('\r\n') #use the \ to seperate the information
        var = subText[1].strip().split(' | ') #use the | to seperate the information
        row['date'] = var[0] #takes first value in var
        row['gender'] = var[1] #takes second value in var
        row['age'] = var[2] if len(var)==3 else np.nan #takes 3rd value in var if it exists otherwise NaN
        match = rx_dose.search(subText[3]) #takes 4th element of subText
        row['dose'] = np.nan if match is None else match.group(1) #takes 2nd value in match if it exists otherwise NaN
        row['disease'] = subText[4].strip() if len(subText)>4 else np.nan 
        span_tag = div_tag.find('span', {'class': 'initial-text'}) #finds up the review in its div_tag
        row['comment'] = span_tag.text if span_tag else ''
        for span_tag in div_tag.findAll('span', {'style': 'width:65%'}): #finds the width % of the 4 little graphs
            text = span_tag.text
            img_tag = span_tag.next_sibling.next_sibling
            row[text] = img_tag.attrs['style'] if img_tag else ''
        rows.append(row)

df = pd.DataFrame(rows)
df['age']=df['age'].astype(float)
df['age_range']=pd.cut(df['age'], [0,20,35,50,65,80,np.inf], labels = ['0-20','21-35','36-50','51-65','66-80','80+'])
df['gender'].replace(gender_dict, inplace=True)
df['disease'].replace(disease_dict, inplace=True)
df["Efficacité"].replace(width_dict, inplace=True)
df["Quantité effets secondaires"].replace(width_dict, inplace=True) 
df["Gravité effets secondaires"].replace(width_dict, inplace=True)
df["Facilité d'emploi"].replace(width_dict, inplace=True)

In [874]:
df = df.rename(columns={"Efficacité": "efficiency", "Quantité effets secondaires": "quantity_of_secondary_effects", 
                   "Gravité effets secondaires": "severity_of_secondary_effects", "Facilité d'emploi": "ease_of_use"})

In [875]:
df.to_csv('methotrexate_review.csv') #store the dataframe in a csv file